<center>
    <h1 style="color:blue">DataBase Connections - Neo4J</h1>
</center>

<p>Tasks:</p>

* Connection
* Insert
* Update
* Select
* Return
* Delete
* Close Connection

<br>

> pip install neo4j

In [1]:
#import neo4j
from neo4j import GraphDatabase

In [4]:
#neo4j.__version__

'4.4.3'

<h2 style="color:blue">0 - Opening Connection</h2>

In [9]:
driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'Sport_GeneratioN1'))

----

<h2 style="color:blue">1 - Functions</h2>

In [18]:
# creating functions

def add_character(tx, character_name, anime_name):
    '''
    Checks whether there is a node created with the same infos:
    - If there is, nothing happens
    - If there is not, the node is created
    '''
    tx.run('MERGE (n:Character {name: $name, anime: $anime});'
          , name=character_name, anime=anime_name)

In [20]:
#def update_character(tx, *args):    
def update_character(tx, character_name, new_name=None, new_anime=None):
    '''
    Updates Character info
    **kwargs will bi assigned with the new character's name and anime
    '''
    
    # Variables
    actual_name = character_name
    n_name = new_name
    n_anime = new_anime
    flag = None
    
    # Defining the Flag:
    # - 0 >> both new name and new anime were assigned
    # - 1 >> just new name was assigned
    # - 2 >> just new anime was assigned
    # - 3 >> both of the optional variables weren't assigned
    flag = None
    
    if (n_name is not None and n_anime is not None): flag = 0
    elif (n_name is not None and n_anime is None): flag = 1
    elif (n_name is None and n_anime is not None): flag = 2
    else: flag = 3
        
    # Executing the Code
    if flag == 0:
        tx.run('MATCH (n:Character {name: $name}) '
               'SET n.name = $new_name, n.anime = $new_anime;'
               , name=actual_name, new_name=n_name, new_anime=n_anime)
        
    elif flag == 1:
        tx.run('MATCH (n:Character {name: $name}) '
               'SET n.name = $new_name;'
               , name=actual_name, new_name=n_name)
    
    elif flag == 2:
        tx.run('MATCH (n:Character {name: $name}) '
               'SET n.anime = $new_anime;'
               , name=actual_name, new_anime=n_anime)
    
    else: print('No datas to update!')

In [34]:
def select_character(tx, character_name):
    '''
    Selects and returns a character
    '''
    for record in tx.run('MATCH (n:Character {name: $name}) RETURN n.name, n.anime;', name=character_name):
        print('Name: ', record['n.name'])
        print('Anime: ', record['n.anime'])
        print()
        print('*-*-*-*-*')
        print()

In [28]:
def delete_character(tx, character_name):
    '''
    Deletes a character from the database
    '''
    tx.run('MATCH (n:Character {name:$name}) '
           'DETACH DELETE (n);'
          , name=character_name)

In [22]:
def delete_all_characters(tx):
    '''
    Deletes all Noodes with Character Label
    '''
    tx.run('MATCH (n:Character) '
           'DETACH DELETE (n);')

----

<h2 style="color:blue">2 - Inserting, Updating, Selecting, Returning and Deleting</h2>

In [38]:
with driver.session() as session:
    
    # inserting four characters
    
    print('Inserting the Characters: Started')
    session.write_transaction(add_character, 'Son Goku', 'Dragon Ball')
    session.write_transaction(add_character, 'Meliodas', 'Nanatsu no Taizai')
    session.write_transaction(add_character, 'Todoroki', 'Boku no Hero Academia')
    session.write_transaction(add_character, 'Arthur', 'Fire Force')
    print('Inserting the Characters: Wrapped Up\n')
    
    # updating the characters
    # - Son Goku: name changed to Kakarot
    # - Meliodas: anime changed to The Four Knights of the Apocalypse
    # - Todoroki: name changed to Joseph and anime changed to JoJo
    # - Arthur: nothing

    print('Updating the Characters: Started')
    session.write_transaction(update_character, 'Son Goku', new_name='Kakarot')
    session.write_transaction(update_character, 'Meliodas', new_anime='The Four Knights of the Apolypse')
    session.write_transaction(update_character, 'Todoroki', new_name='Joseph', new_anime='JoJo')
    session.write_transaction(update_character, 'Arthur')
    print('Updating the Characters: Wrapped Up\n')
    
    # returning all the anime characters in the DataBase
    
    print('Returning the Characters: Started')
    session.read_transaction(select_character, 'Kakarot')
    session.read_transaction(select_character, 'Meliodas')
    session.read_transaction(select_character, 'Joseph')
    session.read_transaction(select_character, 'Arthur')
    print('Returning the Characters: Wrapped Up\n')
    
    # deleting Meliodas from the DataBase
    
    print('Deleting Meliodas: Started')
    session.write_transaction(delete_character, 'Meliodas')
    
    session.read_transaction(select_character, 'Kakarot')
    session.read_transaction(select_character, 'Meliodas')
    session.read_transaction(select_character, 'Joseph')
    session.read_transaction(select_character, 'Arthur')
    print('Deleting Meliodas: Wrapped Up\n')
    
    # deleting all the character in the DataBase
    
    print('Deleting All Remaining Characters: Started')
    session.write_transaction(delete_all_characters)
    
    session.read_transaction(select_character, 'Kakarot')
    session.read_transaction(select_character, 'Meliodas')
    session.read_transaction(select_character, 'Joseph')
    session.read_transaction(select_character, 'Arthur')
    print('Deleting All Remaining Characters: Wrapped Up\n')

Inserting the Characters: Started
Inserting the Characters: Wrapped Up

Updating the Characters: Started
No datas to update!
Updating the Characters: Wrapped Up

Returning the Characters: Started
Name:  Kakarot
Anime:  Dragon Ball

*-*-*-*-*

Name:  Meliodas
Anime:  The Four Knights of the Apolypse

*-*-*-*-*

Name:  Joseph
Anime:  JoJo

*-*-*-*-*

Name:  Arthur
Anime:  Fire Force

*-*-*-*-*

Returning the Characters: Wrapped Up

Deleting Meliodas: Started
Name:  Kakarot
Anime:  Dragon Ball

*-*-*-*-*

Name:  Joseph
Anime:  JoJo

*-*-*-*-*

Name:  Arthur
Anime:  Fire Force

*-*-*-*-*

Deleting Meliodas: Wrapped Up

Deleting All Remaining Characters: Started
Deleting All Remaining Characters: Wrapped Up



----

<h2 style="color:blue">3 - Closing Connection</h2>

In [39]:
driver.close()

<center>
    <img src='thats all.jpg'>
</center>